### 지도학습

#### 분류 - 이진분류
- 목표변수가 2개 항목인 것
- 목표변수 : Survived

In [36]:
import pandas as pd

In [37]:
df_TFD = pd.read_csv('../../../datasets/TitanicFromDisaster_train.csv')
df_TFD[:2]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


#### 전처리

In [38]:
df_TFD.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [39]:
df_TFD_extract = df_TFD[['Survived', 'Pclass', 'Age']]
df_TFD_extract.isnull().sum()

Survived      0
Pclass        0
Age         177
dtype: int64

In [40]:
# age null값 존재 -> regression으로 null값을 채울 수 있음. 여기선 drop함
df_TFD_extract_preprocess = df_TFD_extract.dropna()
df_TFD_extract_preprocess[:2]

,Survived,Pclass,Age
0,0,3,22.0
1,1,1,38.0


#### Scaling & Encoding

##### Encoding with OneHoyEncoding

In [41]:
df_TFD_extract_preprocess['Pclass'].value_counts()

3    355
1    186
2    173
Name: Pclass, dtype: int64

In [42]:
from sklearn.preprocessing import OneHotEncoder

In [43]:
oneHotEncoder = OneHotEncoder()
oneHotEncoder.fit(df_TFD_extract_preprocess[['Pclass']]) # 해당 항목 학습한 것임.

OneHotEncoder()

In [44]:
columns_name = oneHotEncoder.categories_

In [45]:
# oneHotEncoder.transform(df_TFD_extract_preprocess[['Pclass']]).toarray() # 실제값 확인용
encoded_data = oneHotEncoder.transform(df_TFD_extract_preprocess[['Pclass']]).toarray()

In [46]:
encoded_data.shape

(714, 3)

In [47]:
# 병합 위해 numpy array to DataFrame
df_encoded_data = pd.DataFrame(data=encoded_data, columns=oneHotEncoder.get_feature_names_out(['Pclass']))
df_encoded_data[:2]

,Pclass_1,Pclass_2,Pclass_3
0,0.0,0.0,1.0
1,1.0,0.0,0.0


In [48]:
df_encoded_data.index, df_encoded_data.shape

(RangeIndex(start=0, stop=714, step=1), (714, 3))

In [49]:
df_TFD_extract_preprocess.index, df_TFD_extract_preprocess.shape

(Int64Index([  0,   1,   2,   3,   4,   6,   7,   8,   9,  10,
             ...
             880, 881, 882, 883, 884, 885, 886, 887, 889, 890],
            dtype='int64', length=714),
 (714, 3))

In [50]:
# pd.concat([df_TFD_extract_preprocess, df_encoded_data], axis=1, ignore_index=True)
# df_encoded_data = pd.get_dummies(df_TFD_extract_preprocess['Pclass'], prefix='Pclass')

In [51]:
df_TFD_extract_preprocess = pd.concat([df_TFD_extract_preprocess.reset_index(drop=True), df_encoded_data.reset_index(drop=True)], axis=1)
df_TFD_extract_preprocess[:2]

,Survived,Pclass,Age,Pclass_1,Pclass_2,Pclass_3
0,0,3,22.0,0.0,0.0,1.0
1,1,1,38.0,1.0,0.0,0.0


In [56]:
df_TFD_extract_preprocess.shape

(714, 6)

In [52]:
df_TFD_extract_preprocess[['Pclass']][:3]

,Pclass
0,3
1,1
2,3


In [59]:
target = df_TFD_extract_preprocess['Survived']

In [60]:
features = df_TFD_extract_preprocess.drop(columns=['Survived', 'Pclass'])

In [61]:
features.columns

Index(['Age', 'Pclass_1', 'Pclass_2', 'Pclass_3'], dtype='object')

#### MinMaxScaler

In [62]:
from sklearn.preprocessing import MinMaxScaler

In [63]:
minMaxScaler = MinMaxScaler() # 인스턴스화
features = minMaxScaler.fit_transform(features)
features.shape

(714, 4)

#### 정형화 단계 
- 데이터를 머신에 넣기 전 목표변수와 설명변수를 분리
- 데이터 분할
    - 머신러닝 모델을 훈련하고 성능을 측정할 때 훈련 데이터와 테스트 데이터 비율을 8:2로 설정한다.
    - train_test_split() : 8:2 비율로 만들어주는 fuction
    - 데이터가 500개 미만일 경우 split을 하는 것보다 데이터를 더 모으는 게 좋다.

In [64]:
from sklearn.model_selection import train_test_split

In [96]:
features_train, features_test, target_train, target_test = train_test_split(features, target, random_state=111)
features_train.shape, target_train.shape, features_test.shape, target_test.shape

((535, 4), (535,), (179, 4), (179,))

In [66]:
# target_train = df_TFD_extract_preprocess['Survived']
# features_train = df_TFD_extract_preprocess[['Pclass', 'Age']]   # label(=feature)
# target_train.shape, features_train.shape

((714,), (714, 2))

#### 모델학습

In [112]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()  # 인스턴스화
model.fit(features_train, target_train)  # 모델 훈련은 fit() fuction을 이용함 -> fit(feature, target)

LogisticRegression()

In [113]:
model.coef_, model.intercept_

(array([[-2.59255724,  1.11141847,  0.02997489, -1.14146766]]),
 array([0.86447724]))

#### 예측

In [114]:
# 값을 넣어서 확인해보기
df_TFD_extract_preprocess[10:15]   # index가 features_train과 같다.

,Survived,Pclass,Age,Pclass_1,Pclass_2,Pclass_3
10,1,1,58.0,1.0,0.0,0.0
11,0,3,20.0,0.0,0.0,1.0
12,0,3,39.0,0.0,0.0,1.0
13,0,3,14.0,0.0,0.0,1.0
14,1,2,55.0,0.0,1.0,0.0


In [115]:
model.predict(features_train[10:15])
# 위에서는 0이 3개 1이 2개
# 결과: array([0, 0, 0, 0, 0]) 60% 맞음

array([0, 0, 0, 1, 0], dtype=int64)

In [116]:
# predict_proba() : 확률을 알 수 있음
model.predict_proba(features_train[10:15])

# [0.52507531, 0.47492469] -> 앞은 0에 대한 열, 뒤는 1에 대한 열
# 0에 대한 확률이 더 높기 때문에 위에서 0으로 결과가 나온 것

array([[0.59715375, 0.40284625],
       [0.63564456, 0.36435544],
       [0.75219168, 0.24780832],
       [0.47500723, 0.52499277],
       [0.73984819, 0.26015181]])

#### 평가

In [117]:
target_train_predict = model.predict(features_train)
target_train_predict.shape  # target_train.shape과 동일

(535,)

In [118]:

from sklearn.metrics import accuracy_score

In [119]:
accuracy_score(target_train, target_train_predict) # 교내 시험
# 0.7065420560747664 정규화 이전

0.708411214953271

In [120]:
target_test_perdict = model.predict(features_test)
target_test_perdict.shape # target_test.shape 동일

(179,)

In [121]:
accuracy_score(target_test, target_test_predict)   # 교외 시험

# 교내 시험, 교외 시험의 두 개의 차이가 많이 나면 모델 성능에 문제가 있다고 판단
# 0.7, 0.65는 0.05정도의 차이이므로 양호함

NameError: name 'target_test_predict' is not defined

##### 성능지표
1. accuracy(정확도)
2. precision(정밀도) : 모델의 예측결과가 positive인 것 중 실제값이 positive인 확률
3. recall(재현율) : 실제 True인 것 중에서 모델이 True라고 예츠한 것의 비율로 민감도와 같은 의미
    - 정밀도와 재현율은 상호보완적, 두 지표가 모두 높을수록 좋은 지표

In [77]:
from sklearn.metrics import classification_report

In [78]:
print(classification_report(target_train, target_train_predict))   # vs code의 파이썬에서 사용되는 코드라서 print로 감싸줘야 모양이 이상하지 않게 나옴

              precision    recall  f1-score   support

           0       0.71      0.82      0.76       424
           1       0.66      0.52      0.58       290

    accuracy                           0.70       714
   macro avg       0.69      0.67      0.67       714
weighted avg       0.69      0.70      0.69       714



In [110]:
print(classification_report(target_test, target_test_predict))

NameError: name 'target_test_predict' is not defined

#### 오차 행렬

In [122]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

In [123]:
confusion_matrix(target_train, target_train_predict)

array([[257,  55],
       [101, 122]], dtype=int64)

In [124]:
precision_score(target_train, target_train_predict)

0.6892655367231638

In [125]:
recall_score(target_train, target_train_predict)

0.547085201793722

In [126]:
f1_score(target_train, target_train_predict)

0.6100000000000001

#### 서비스

In [85]:
df_TFD_extract[100:103]

,Survived,Pclass,Age
100,0,3,28.0
101,0,3,NaN
102,0,1,21.0


In [89]:
# 예측 대상자 입력값 Pclass : 3, Age : 28.0
encoder_Pclass_ = oneHotEncoder.transform([[3]]).toarray()
encoder_Pclass_, encoder_Pclass_.flatten()

C:\Users\01-02\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


(array([[0., 0., 1.]]), array([0., 0., 1.]))

In [91]:
# [[28.0, encoder_Pclass_]]
import numpy as np
inputs_data = np.concatenate(([28.0], encoder_Pclass_.flatten()))
inputs_data

array([28.,  0.,  0.,  1.])

In [111]:
model.predict([inputs_data])  # 목표변수 얻기

array([0], dtype=int64)